In [8]:
import csv
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Set up the web driver (make sure to provide the path to your WebDriver executable)
# Choose the ChromeDriver based on your operating system
service = Service(executable_path='./chromedriver-win64/chromedriver.exe')
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)

# URL to scrape
url = "http://freesound.org"
keywords = "fox" # Replace the key word if you want to search for different sounds

# Function to scrape and save weather audio clips
def scrape_weather_audio():
    driver.get(url)
    time.sleep(2)  # Wait for the page to load

    # Find the search bar and enter the search query (e.g., "weather")
    search_bar = WebDriverWait(driver, 20).until(
    EC.element_to_be_clickable((By.ID, "search-sounds"))
    )
    search_bar.send_keys(keywords)
    search_bar.send_keys(Keys.RETURN)

    # Wait for search results to load
    time.sleep(10)

    # Get page source and parse with BeautifulSoup
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, "html.parser")

    # Find the audio clips on the page
    audio_clips = soup.find_all("div", class_="sample_player_small")

    # Create a list to store audio clip information
    audio_info = []

    # Loop through each audio clip and collect information
    for clip in audio_clips:
        clip_name = clip.find("div", class_="sample_player").find("div", class_="sound_title").find("div", class_="sound_filename").text.strip()
        clip_info = clip.find("div", class_="sample_information")
        clip_uploaded_by = clip_info.find("a", class_="user").text.strip()
        #clip_length = clip.find("div", class_="sample_player").find("div", class_="small_player").find("div", class_="player small").find("div", class_="time-indicator-container").find("div", class_="time-indicator").text.strip()
        clip_date = clip_info.find("span", class_="date").text.strip()
        clip_download = clip_info.find("span", class_="download_count").text.strip()
        
        audio_info.append({
            "Name": clip_name,
            "Uploaded By": clip_uploaded_by,
            #"Length": clip_length,
            "Date": clip_date,
            "Download": clip_download
        })

    # Extract audio clip information
    # audio_clips = []
    # for item in soup.find_all("div", class_="audio-item"):
    #     name = item.find("div", class_="title").get_text()
    #     uploaded_by = item.find("div", class_="username").get_text()
    #     length = item.find("div", class_="duration").get_text()
    #     audio_clips.append({"name": name, "uploaded_by": uploaded_by, "length": length})


    return audio_info

# Scrape audio clips and save them in a CSV file
def save_to_csv(data):
    with open(keywords+"_audio_clips.csv", "w", newline="", encoding="utf-8") as csvfile:
        fieldnames = ["Name", "Uploaded By", "Date", "Download"]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for clip in data:
            writer.writerow(clip)

if __name__ == "__main__":
    try:
        audio_data = scrape_weather_audio()
        save_to_csv(audio_data)
        print("Scraping and CSV creation successful.")
    except Exception as e:
        print(f"An error occurred: {e.__class__.__name__}, {str(e)}")
    finally:
        driver.quit()


Scraping and CSV creation successful.


In [9]:
import requests
import os

 
# Apply for the api key on https://freesound.org/help/developers/
API_KEY = "Your Freesound API key"  # Replace with your API key
BASE_URL = "https://freesound.org/apiv2"
DOWNLOAD_DIR = "Download_path" # Change this path to your local folder
PAGESIZE = 15
 
# search sounds in Freesounds
def search_freesound(query, limit=PAGESIZE):
    endpoint = f"{BASE_URL}/search/text/"
    params = {
        "query": query,
        "token": API_KEY,
        "fields": "id,name,previews",
        "page_size": limit
    }
    response = requests.get(endpoint, params=params)
    response.raise_for_status()  # Raise an error for bad responses
    return response.json()['results']

 
# Download a sound from Freesound.
def download_sound(sound_id, preview_url, filename):
    if not os.path.exists(DOWNLOAD_DIR):
        os.makedirs(DOWNLOAD_DIR)

    response = requests.get(preview_url)
    response.raise_for_status()

    with open(os.path.join(DOWNLOAD_DIR, filename), 'wb') as f:
        f.write(response.content)
    print(f"Downloaded {filename}")

 

def main():
    query = keywords
    results = search_freesound(query)
    for result in results:
        sound_id = result['id']
        preview_url = result['previews']['preview-hq-mp3']
        filename = f"{result['name']}.wav"
        download_sound(sound_id, preview_url, filename)

 

if __name__ == "__main__":
    main()

Downloaded red fox screeching.mp3.wav
Downloaded fox_barks_cut.wav.wav
Downloaded 25-fox eats.wav.wav
Downloaded Amtor FEC-Navtex 3 x the quick brown fox.wav.wav
Downloaded Fox sound in the field.wav
Downloaded rtty 75 bd testslip the quick brown fox and 1 row ry.wav.wav


KeyboardInterrupt: 